# Neighborlist Feature Generators <a name="head"></a>

In this tutorial, we will look at generating features from a database of organic donor-acceptor molecules from the [Computational Materials Repository](https://cmrdb.fysik.dtu.dk/?project=solar). This has been downloaded in the [ase-db](https://wiki.fysik.dtu.dk/ase/ase/db/db.html#module-ase.db) format so first off we load the atoms objects and get a target property. Then we convert the atoms objects into a feature array and test out a couple of different models.

This tutorial will give an indication of one way in which it is possible to handle atoms objects of different sizes. In particular, we focus on a feature set that scales with the number of atoms. We pad the feature vectors to a constant size to overcome this problem.

In [ ]:
%matplotlib inline
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.font_manager as font_manager
import pandas as pd
from pandas.plotting import parallel_coordinates
import seaborn as sns

import ase.db

from atoml.fingerprint.setup import FeatureGenerator
from atoml.regression import RidgeRegression, GaussianProcess
from atoml.cross_validation import Hierarchy
from atoml.regression.cost_function import get_error

In [ ]:
# Connect the ase-db.
db = ase.db.connect('../../data/solar.db')
atoms = list(db.select())
random.shuffle(atoms)

# Compile a list of atoms and target values.
alist = []
targets = []
for row in atoms:
    try:
        targets.append(row.Energy)
        alist.append(row.toatoms())
    except AttributeError:
        continue

# Analyze the size of molecules in the db.
print('pulled {} molecules from db'.format(len(alist)))
size = []
for a in alist:
    size.append(len(a))

print('min: {0}, mean: {1:.0f}, max: {2} molecule size'.format(
    min(size), sum(size)/len(size), max(size)))

In [ ]:
from atoml.fingerprint.neighbor_matrix import neighbor_features

r = []
f = []
for a in alist[:10]:
    res = neighbor_features(a, property=['en_pauling', 'vdw_radius', 'atomic_number'])
    r.append(len(res))
    f.append(res)

In [ ]:
max_vec = max(r)

norm = []
for i in f:
    i = list(i)
    if len(i) < max_vec:
        i += [0] * (max_vec - len(i))
    norm.append(i)

print(np.shape(norm))

train_features = np.asarray(norm)[:5, :]
test_features = np.asarray(norm)[5:10, :]
train_targets = np.asarray(targets)[:5]
test_targets = np.asarray(targets)[5:10]

In [ ]:
# Set up the ridge regression function.
rr = RidgeRegression(W2=None, Vh=None, cv='loocv')
b = rr.find_optimal_regularization(X=train_features, Y=train_targets)
coef = rr.RR(X=train_features, Y=train_targets, omega2=b)[0]

# Test the model.
sumd = 0.
err = []
pred = []
for tf, tt in zip(test_features, test_targets):
    p = np.dot(coef, tf)
    pred.append(p)
    sumd += (p - tt) ** 2
    e = ((p - tt) ** 2) ** 0.5
    err.append(e)
error = (sumd / len(test_features)) ** 0.5

print(error)

plt.figure(figsize=(30, 15))
plt.plot(test_targets, pred, 'o', c='b', alpha=0.5)
plt.savefig('no_geom.png')

In [ ]:
kdict = {
    'k1': {
        'type': 'gaussian', 'width': 1., 'scaling': 1., 'dimension': 'single'},
    'k2' : {
        'type': 'linear', 'scaling': 1.},
    }
gp = GaussianProcess(train_fp=train_features, train_target=train_targets,
                     kernel_dict=kdict, regularization=1e-2,
                     optimize_hyperparameters=True, scale_data=True)

pred = gp.predict(test_fp=test_features)

error = get_error(pred['prediction'],
                  test_targets)['rmse_average']

print(error)

plt.figure(figsize=(30, 15))
plt.plot(test_targets, pred['prediction'], 'o', c='r', alpha=0.5)
plt.savefig('no_geom_gp.png')